In [1]:
import numpy as np
import pandas as pd
from pandas import Series
import json
from pandas import json_normalize
import time
df_result = pd.read_csv('data/df_result.csv',low_memory=False, index_col = 0)
df_result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,frozenset({'ASSETS_VAL_1_60'}),frozenset({'101.231.133.165'}),0.561955,0.001190,0.001114,0.001982,1.664695,0.000445,1.000793
1,frozenset({'101.231.133.165'}),frozenset({'ASSETS_VAL_1_60'}),0.001190,0.561955,0.001114,0.935484,1.664695,0.000445,6.789694
2,frozenset({'ASSETS_VAL_1_50'}),frozenset({'103.139.2.240'}),0.322851,0.002304,0.002112,0.006541,2.839290,0.001368,1.004265
3,frozenset({'103.139.2.240'}),frozenset({'ASSETS_VAL_1_50'}),0.002304,0.322851,0.002112,0.916667,2.839290,0.001368,8.125792
4,frozenset({'103.233.103.20'}),frozenset({'ASSETS_VAL_1_60'}),0.001459,0.561955,0.001229,0.842105,1.498527,0.000409,2.774284
...,...,...,...,...,...,...,...,...,...
289,"frozenset({'ASSETS_VAL_5_100', 'ASSETS_VAL_4_1...","frozenset({'ASSETS_VAL_3_100', 'ASSETS_VAL_1_6...",0.003417,0.121223,0.003379,0.988764,8.156599,0.002965,78.211189
290,frozenset({'ASSETS_VAL_3_100'}),"frozenset({'ASSETS_VAL_4_100', 'ASSETS_VAL_5_1...",0.122567,0.003417,0.003379,0.027569,8.067162,0.002960,1.024836
291,frozenset({'ASSETS_VAL_1_60'}),"frozenset({'ASSETS_VAL_3_100', 'ASSETS_VAL_5_1...",0.561955,0.003379,0.003379,0.006013,1.779501,0.001480,1.002650
292,frozenset({'ASSETS_VAL_5_100'}),"frozenset({'ASSETS_VAL_3_100', 'ASSETS_VAL_4_1...",0.005990,0.006720,0.003379,0.564103,83.948132,0.003339,2.278702


In [2]:
#필요한 라이브러리 설치
!pip install elasticsearch

In [3]:
#필요한 패키지 가져오고 파이썬에서 엘라스틱 연결
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [4]:
list(df_result.columns.values)

['antecedents',
 'consequents',
 'antecedent support',
 'consequent support',
 'support',
 'confidence',
 'lift',
 'leverage',
 'conviction']

In [5]:
df_new = df_result.copy()[['antecedents','consequents','support','confidence','lift']]

In [6]:
#frozenset to string
df_result["antecedents"] = df_result["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
df_result["consequents"] = df_result["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [7]:
import random
import string

In [10]:
#함수를 실행할 때 마다 Index에 데이터가 추가됨. 실행 전 기존 Index 내부 데이터 삭제
es.delete_by_query(index='es_res', doc_type="_doc", body='{"query":{"match_all":{}}}')

{'took': 41,
 'timed_out': False,
 'total': 294,
 'deleted': 294,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [11]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        try:
            yield{
                "_index" : 'es_res',
                "_type" : "_doc",
                "_id" : "".join([random.choice(string.ascii_letters) for _ in range(10)]),
                "_source":filterKeys(document),
            }
        except StopIteration:
            return
    
use_these_keys = list(df_new.columns.values)

def filterKeys(document):
    return {key: document[key] for key in use_these_keys}

helpers.bulk(es, doc_generator(df_new))

(294, [])